# <font size=35 color=lightgreen>** Sentiment API **<font>🥲
---

### <font size=12 color=lightgreen>Configuración Inicial (Librerías)</font>

#### 1. Procesamiento y Manipulación de Datos
* **`pandas`**
    * Nos ayuda con la manipulación y análisis de datos estructurados.
    * Carga el dataset (CSV), gestiona el DataFrame y permite filtrar o limpiar registros.
* **`numpy`**
    * Realiza las operaciones matemáticas y manejo de arrays eficientes.
    * Soporte numérico fundamental para las transformaciones vectoriales de los textos.

#### 2. Visualización y Análisis Exploratorio

* **`matplotlib.pyplot`**
    * Generación de gráficos estáticos.
    * Visualización básica de la distribución de clases (Positivo vs. Negativo).
* **`seaborn`**
    * Visualización de datos estadísticos avanzada.
    * Generación de matrices de confusión y gráficos de distribución estéticos para la presentación.

#### 3. Procesamiento de Lenguaje Natural (NLP) y Limpieza

* **`re`** (Regular Expressions)
    * Manejo de expresiones regulares.
    * Eliminación de ruido en el texto: URLs, menciones (@usuario), hashtags (#) y caracteres especiales no alfanuméricos.
* **`string`**
    * Constantes de cadenas comunes.
    * Provee listas estándar de signos de puntuación para su eliminación eficiente.

#### 4. Modelado y Machine Learning (Core)

* **`scikit-learn`**
    * Biblioteca principal de Machine Learning.
    * **`TfidfVectorizer`**: Transforma el texto limpio en vectores numéricos.
    * **`LogisticRegression`**: Algoritmo de clasificación supervisada.
    * **`metrics`**: Cálculo de precisión, recall y F1-score.
    * **`Pipeline`**: Encapsulamiento de los pasos de transformación y predicción.

#### 5. Persistencia e Integración
Herramientas para conectar el modelo con el Backend.

* **`joblib`**
    * Serialización eficiente de objetos Python.
    * Exportar (`dump`) el pipeline entrenado a un archivo `.joblib` y cargarlo (`load`) en la API para realizar predicciones.
* **`fastapi` & `uvicorn`**
    * Framework web moderno de alto rendimiento.
    * Exponer el modelo entrenado como un microservicio REST (endpoint `/predict`) para ser consumido por el Backend en Java.




---



### <font size=16  color=lightgreen> Importando librerías <font>



In [52]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px
import seaborn as sns
import re
import string
import chardet
import uvicorn
import sklearn
import fastapi
import joblib
import nltk
import unicodedata
import urllib.request
from io import StringIO
import urllib.response
import os
from pathlib import Path
from datetime import datetime
import warnings


### <font size = 8 color="lightgreen">Importación de los datasets<font>

#### **Función importación dataset**

In [53]:
def importar_dataset(url):
    """
    Importa dataset desde URL detectando encoding automáticamente.
    """
    try:
        # 1. Descargar contenido una sola vez
        with urllib.request.urlopen(url) as response:
            content = response.read()

        # 2. Detectar encoding
        result = chardet.detect(content)
        encoding = result['encoding']
        print(f"🔍 Encoding detectado: {encoding} (confianza: {result['confidence']:.2%})")

        # 3. Decodificar y cargar en DataFrame
        decoded_content = content.decode(encoding, errors='replace')
        data = pd.read_csv(StringIO(decoded_content), sep=';')

        print("✅ Archivo cargado correctamente")
        print(f"📊 Tamaño del dataset: {data.shape}")
        print("\n🔍 Muestra aleatoria (3 registros):")
        print(data.sample(3))

        return data

    except urllib.error.URLError as e:
        print(f"❌ Error de URL: {e}")
        return None
    except pd.errors.ParserError as e:
        print(f"❌ Error al parsear CSV: {e}")
        return None
    except Exception as e:
        print(f"❌ Error inesperado: {type(e).__name__}: {e}")
        return None

#### **Dataset1: sentimentdataset_es.csv**

In [54]:
df1_raw = importar_dataset("https://github.com/ml-punto-tech/sentiment-api/raw/refs/heads/dev/data-science/datasets/datasets-origin/sentimentdataset_es.csv")


🔍 Encoding detectado: Windows-1252 (confianza: 72.97%)
✅ Archivo cargado correctamente
📊 Tamaño del dataset: (732, 15)

🔍 Muestra aleatoria (3 registros):
     Unnamed: 0.1  Unnamed: 0  \
257           543         547   
600            15          15   
331           665         669   

                                                  Text   Sentiment  \
257  En el campeonato de cricket, un final emociona...  Excitación   
600  Reflexionando sobre el pasado y mirando hacia ...    Positivo   
331  Esconder un refrigerio en mi mochila para los ...    Positivo   

            Timestamp                           User  Platform  \
257  25-06-2019 22:15  CricketFanChampionshipThrills  Facebook   
600  19-01-2023 13:20                    Reflections  Facebook   
331  23-08-2023 16:45           SnackNinjaHighSchool   Twitter   

                             Hashtags  Retweets  Likes Country  Year  Month  \
257   #Emoción #Campeonato de Cricket        30     60   India  2019      6   
600     

#### **Dataset2: sentiment_analysis_dataset.csv**

In [55]:
df2_raw = importar_dataset("https://raw.githubusercontent.com/ml-punto-tech/sentiment-api/refs/heads/feature/data-science-marely/data-science/datasets/datasets-origin/sentiment_analysis_dataset.csv")

🔍 Encoding detectado: Windows-1252 (confianza: 73.00%)
✅ Archivo cargado correctamente
📊 Tamaño del dataset: (2540, 3)

🔍 Muestra aleatoria (3 registros):
                                                 texto  label sentimiento
65         Desbordado de felicidad y de ansiedad??????      0    negativo
644  Ése es el Licey, se crecen los que les duele é...      0    negativo
163  Insólito, la oposición está mortificada porque...      0    negativo


In [56]:
def verificar_calidad_importacion(df, nombre_dataset):
    """
    Verifica que no se haya perdido información durante la importación.
    """
    print(f"\n🔍 VERIFICACIÓN DE CALIDAD: {nombre_dataset}")
    print("=" * 60)

    if df is None:
        print("❌ Dataset es None")
        return False

    # 1. Información básica
    print(f"📊 Forma: {df.shape}")
    print(f"📝 Columnas: {list(df.columns)}")

    # 2. Buscar columnas de texto
    columnas_texto = [col for col in df.columns if df[col].dtype == 'object']
    print(f"🔤 Columnas de texto: {columnas_texto}")

    if not columnas_texto:
        print("⚠️  No se encontraron columnas de texto")
        return True

    # 3. Analizar una columna de texto (usar la primera)
    col_texto = columnas_texto[0]
    print(f"\n📝 Analizando columna: '{col_texto}'")

    # Muestra de textos
    textos = df[col_texto].dropna().head(5).tolist()

    problemas = []

    for i, texto in enumerate(textos):
        if isinstance(texto, str):
            # Buscar caracteres de reemplazo (�) que indican problemas
            caracteres_problema = texto.count('�')
            if caracteres_problema > 0:
                problemas.append(f"Texto {i+1} tiene {caracteres_problema} caracteres de reemplazo (�)")

            # Buscar emojis
            emojis = [c for c in texto if unicodedata.category(c)[0] in ['S', 'So']]
            if emojis:
                print(f"  Texto {i+1}: ✅ Tiene {len(emojis)} emoji(s): {''.join(emojis[:3])}")
            else:
                print(f"  Texto {i+1}: 📄 Sin emojis")

            # Mostrar fragmento
            preview = texto[:80] + "..." if len(texto) > 80 else texto
            print(f"     '{preview}'")

    # 4. Resumen
    if problemas:
        print(f"\n⚠️  PROBLEMAS ENCONTRADOS:")
        for problema in problemas:
            print(f"   • {problema}")
        return False
    else:
        print(f"\n✅ CALIDAD OK: No se detectaron caracteres perdidos")
        return True

verificar_calidad_importacion(df1_raw, "Dataset 1")
verificar_calidad_importacion(df2_raw, "Dataset 2")


🔍 VERIFICACIÓN DE CALIDAD: Dataset 1
📊 Forma: (732, 15)
📝 Columnas: ['Unnamed: 0.1', 'Unnamed: 0', 'Text', 'Sentiment', 'Timestamp', 'User', 'Platform', 'Hashtags', 'Retweets', 'Likes', 'Country', 'Year', 'Month', 'Day', 'Hour']
🔤 Columnas de texto: ['Text', 'Sentiment', 'Timestamp', 'User', 'Platform', 'Hashtags', 'Country']

📝 Analizando columna: 'Text'
  Texto 1: 📄 Sin emojis
     '¡Acabo de adoptar a un lindo amigo peludo!??'
  Texto 2: 📄 Sin emojis
     '¡Acabo de terminar un entrenamiento increíble!??'
  Texto 3: 📄 Sin emojis
     '¡Adoración desbordante por un lindo cachorro rescatado!??'
  Texto 4: 📄 Sin emojis
     '¡Año nuevo, nuevos objetivos de fitness!??'
  Texto 5: 📄 Sin emojis
     '¡Celebrando el cumpleaños de un amigo esta noche!??'

✅ CALIDAD OK: No se detectaron caracteres perdidos

🔍 VERIFICACIÓN DE CALIDAD: Dataset 2
📊 Forma: (2540, 3)
📝 Columnas: ['texto', 'label', 'sentimiento']
🔤 Columnas de texto: ['texto', 'sentimiento']

📝 Analizando columna: 'texto'
  Texto

True

<font color='lightgreen' size=12>Filtrar datasets</font>

In [57]:
# Función filtrar dataset
def filtrar_dataset(data):
    data_filtro = data[['texto', 'sentimiento']]
    data_filtro = data_filtro[data_filtro['texto'].str.strip() != ""]
    print(data_filtro.sample(5))
    return data_filtro

# Reemplazar nombre columnas Text por texto, Sentiment por sentimiento
df1_raw.rename({'Text':'texto', 'Sentiment':'sentimiento'}, axis=1, inplace=True)
df1_filtrado = filtrar_dataset(df1_raw)
df2_filtrado = filtrar_dataset(df2_raw)

                                                 texto sentimiento
194   Despertar de entusiasmo por un próximo festival.  Excitación
687  Traición, una serpiente venenosa que se desliz...    Traición
556  Pasando el día con aire de indiferencia, indif...     Neutral
123  Balanceándose al ritmo de las melodías acústic...   Serenidad
317  Entusiasmo por un nuevo proyecto artístico en ...  Entusiasmo
                                                  texto sentimiento
842   con algunas personas perdemos el tiempo con ot...    negativo
2110  A veces uno se siente desconcertado,porque no ...     neutral
1004  Serían yo somnoliento diciéndole al despertado...    negativo
1127  Quiero renunciar solo para tener mi liquidació...    negativo
290   Por esta vida de mierda  Gracias  Gracias  Gra...    negativo


### <font size= 12 color="lightgreen" >Explorando los datasets<font>

In [58]:
# Crear función para explorar datasets
def explorar_dataset(data):
    print('Filas: ' + str(data.shape[0]))
    print('Columnas: ' + str(data.shape[1]))
    print('\nColumnas: \n' + str(data.columns.tolist()))
    print('\nTipo de datos: \n' + str(data.dtypes))
    print('\nValores nulos: \n' + str(data.isnull().sum()))
    print('\nMuestra aleatoria (5 registros): \n' + str(data.sample(5)))

#### **Explorando Data1**

In [59]:
explorar_dataset(df1_filtrado)

Filas: 732
Columnas: 2

Columnas: 
['texto', 'sentimiento']

Tipo de datos: 
texto          object
sentimiento    object
dtype: object

Valores nulos: 
texto          0
sentimiento    0
dtype: int64

Muestra aleatoria (5 registros): 
                                                 texto sentimiento
649   Sentirse realizado después de un día productivo.    Positivo
34   Abrazar los defectos, encontrar aceptación en ...  Aceptación
281  En la cima de Machu Picchu, un panorama impres...    Positivo
466  La oscuridad desciende y envuelve el alma en l...     Neutral
113  Asombro ante la belleza de un doble arcoíris d...  Admiración


#### **Explorando data2**

In [60]:
explorar_dataset(df1_filtrado)

Filas: 732
Columnas: 2

Columnas: 
['texto', 'sentimiento']

Tipo de datos: 
texto          object
sentimiento    object
dtype: object

Valores nulos: 
texto          0
sentimiento    0
dtype: int64

Muestra aleatoria (5 registros): 
                                                 texto sentimiento
620  Rodeado de los colores vibrantes de un jardín ...    Positivo
375  Fue testigo de una acalorada discusión en la c...       Odiar
44   Admirando la dedicación de los voluntarios en ...  Admiración
666    Sumergirse con entusiasmo en un nuevo proyecto.  Entusiasmo
434  La confianza se dispara después de superar la ...   Confianza


### <font size=12 color=lightgreen>Limpiar textos</font>

#### **Función para limpieza de textos**

In [61]:
def limpiar_texto_sentimientos(texto):
    """
    Normaliza texto español preservando ñ y eliminando tildes.
    NO convierte a minúsculas para preservar intensidad emocional.
    """
    # Verifica si la entrada no es una cadena. Si no lo es, devuelve una cadena vacía.
    if not isinstance(texto, str):
        return ""

    # 1. Normaliza el texto para separar los caracteres base de sus diacríticos (ej., tildes).
    texto = unicodedata.normalize('NFD', texto)

    # 2. Reemplaza temporalmente las 'ñ' y 'Ñ' con marcadores especiales para preservarlas
    # durante la eliminación de diacríticos.
    texto = texto.replace('n\u0303', '@@@N_TILDE@@@')
    texto = texto.replace('ñ', '@@@N_TILDE@@@')
    texto = texto.replace('N\u0303', '@@@N_TILDE_MAYUS@@@')
    texto = texto.replace('Ñ', '@@@N_TILDE_MAYUS@@@')

    # 3. Elimina los caracteres diacríticos (como las tildes) del texto.
    texto = ''.join(
        char for char in texto
        if not unicodedata.combining(char)
    )

    # Restaura las 'ñ' y 'Ñ' utilizando los marcadores temporales.
    texto = texto.replace('@@@N_TILDE@@@', 'ñ')
    texto = texto.replace('@@@N_TILDE_MAYUS@@@', 'Ñ')

    # Variable para almacenar el resultado de la limpieza.
    resultado = texto
    chars = []

    # Itera sobre cada caracter en el resultado y añade solo los caracteres imprimibles a una lista.
    # Los caracteres no imprimibles (como los de control) son reemplazados por un espacio.
    for char in resultado:
        if char.isprintable():
            chars.append(char)
        else:
            chars.append(' ')
    resultado = ''.join(chars)

    # Elimina URLs que terminan en "..." (posibles URLs rotas).
    resultado = re.sub(r'https?://[^\s]*\.\.\.', '[URL_ROTA]', resultado)
    resultado = re.sub(r'www\.[^\s]*\\.\\.\\.', '[URL_ROTA]', resultado)

    # Normaliza los espacios múltiples a uno solo y elimina espacios al inicio y final.
    resultado = ' '.join(resultado.split())
    resultado = resultado.strip()

    # Mostrar resultados estadísticos de la limpieza.




    # Devuelve el texto preprocesado.
    return resultado


#### **Análisis proceso de limpieza de textos**

In [62]:
def analizar_limpieza_sentimientos(df_antes, df_despues, nombre):
    """
    Análisis específico para tu función limpiar_texto_para_sentimientos
    """
    print(f"\n🔍 ANÁLISIS ESPECÍFICO: {nombre}")
    print("="*60)

    # 1. Cambios en caracteres específicos del español
    cambios_especificos = {
        'tildes_eliminadas': 0,
        'ñ_preservadas': 0,
        'urls_eliminadas': 0,
        'mayusculas_preservadas': 0
    }

    # Muestra de 50 textos para análisis detallado
    muestra = min(50, len(df_antes))

    for i in range(muestra):
        if i < len(df_despues):
            texto_antes = str(df_antes.iloc[i]['texto'])
            texto_despues = str(df_despues.iloc[i]['texto'])

            # Contar ñ preservadas
            if 'ñ' in texto_antes.lower() and 'ñ' in texto_despues.lower():
                cambios_especificos['ñ_preservadas'] += 1

            # Contar URLs eliminadas
            import re
            urls_antes = len(re.findall(r'https?://\S+', texto_antes))
            urls_despues = len(re.findall(r'https?://\S+', texto_despues))
            if urls_antes > urls_despues:
                cambios_especificos['urls_eliminadas'] += (urls_antes - urls_despues)

            # Verificar mayúsculas preservadas
            mayus_antes = sum(1 for c in texto_antes if c.isupper())
            mayus_despues = sum(1 for c in texto_despues if c.isupper())
            if mayus_antes > 0 and mayus_despues > 0:
                cambios_especificos['mayusculas_preservadas'] += 1

    print("📊 Cambios específicos de tu limpiador:")
    for cambio, cantidad in cambios_especificos.items():
        print(f"   • {cambio.replace('_', ' ').title()}: {cantidad} de {muestra} textos")



    print("="*60)

In [63]:
# Lista de dataframes para procesar
dataframes = [
    (df1_filtrado, "Dataset 1"),
    (df2_filtrado, "Dataset 2")
]

resultados = {}

for df, nombre in dataframes:
    # Aplicar limpieza
    df['Texto_Limpio'] = df['texto'].apply(limpiar_texto_sentimientos)

    # Guardar copia limpia
    resultados[nombre] = df.copy()

    # Mostrar info
    print(f"\n📁 {nombre}")
    print(f"   Registros: {len(df):,}")
    print(f"   Muestra (3 textos):")
    print(df[['texto', 'Texto_Limpio']].sample(3))

# Asignar a variables originales
df1_clean = resultados["Dataset 1"]
df2_clean = resultados["Dataset 2"]

analizar_limpieza_sentimientos(df1_filtrado, df1_clean, "Dataset 1")
analizar_limpieza_sentimientos(df2_filtrado, df2_clean, "Dataset 2")


📁 Dataset 1
   Registros: 732
   Muestra (3 textos):
                                                 texto  \
308  Enfrentando el rechazo de la universidad de su...   
189  Deleite caprichoso en un mundo de cuentos de h...   
275  En el tapiz de la desesperación, se desenredan...   

                                          Texto_Limpio  
308  Enfrentando el rechazo de la universidad de su...  
189  Deleite caprichoso en un mundo de cuentos de h...  
275  En el tapiz de la desesperacion, se desenredan...  

📁 Dataset 2
   Registros: 2,540
   Muestra (3 textos):
                                                  texto  \
877   Donde se que lo de los rituales de Epstein son...   
1572  Yo soy la más feliz cada ves que voy a ver a m...   
537   Dios que sensible e irritable ando últimamente...   

                                           Texto_Limpio  
877   Donde se que lo de los rituales de Epstein son...  
1572  Yo soy la mas feliz cada ves que voy a ver a m...  
537   Dios que sen

In [64]:
df1_clean.sample(3)

,texto,sentimiento,Texto_Limpio
703,Un momento de Verguenza por no denunciar la in...,Lástima,Un momento de Verguenza por no denunciar la in...
457,"La gratitud como estrella guía, navegando por ...",Positivo,"La gratitud como estrella guia, navegando por ..."
560,Pasé horas creando la lista de reproducción pe...,Alegría,Pase horas creando la lista de reproduccion pe...


In [65]:
df2_clean.sample(3)

,texto,sentimiento,Texto_Limpio
2396,ver lo de shibuya animado de jujutsu kaisen es...,positivo,ver lo de shibuya animado de jujutsu kaisen es...
1826,"Durante carnavales no tendré ig, decidido",neutral,"Durante carnavales no tendre ig, decidido"
141,"Para su gran amiga @lambofyah, Neville le exte...",negativo,"Para su gran amiga @lambofyah, Neville le exte..."


### <font size=12 color=lightgreen>Categorizar de sentimientos </font>

#### **Categorías Sentimientos**

In [66]:
# 1. Definimos las listas de sentimientos según su categoría
# Ver todos los sentimientos únicos para saber qué agrupar y ordenar alfabetizamente
sentimientos_unicos = sorted(df1_clean['sentimiento'].unique())
print(f"Total de sentimientos únicos: {len(sentimientos_unicos)}")
print(sentimientos_unicos)

# 2. SENTIMIENTOS POSITIVOS COMPLETOS (Bienestar, éxito, alegría, admiración)
positivos = [
    'Aceptación', 'Admiración', 'Adoración', 'Agradecido', 'Alegría', 'Amabilidad', 'Amor', 'Amistad', 'Apreciación', 'Armonía', 'Asombro', 'Cautivación', 'Celebración', 'Colorido', 'Confiado','Confianza', 'Contentamiento', 'Creatividad', 'Cumplimiento', 'Descubrimiento', 'Deslumbrar', 'Determinación', 'Disfrute','Diversión', 'Elegancia', 'Emoción', 'Empático', 'Empoderamiento',
    'Encantamiento', 'Energía', 'Entusiasmo', 'Esperanza', 'Euforia', 'Excitación', 'Felicidad', 'Grandeza', 'Gratitud', 'Inspiración', 'Inspirado', 'Intimidación', 'Juguetón', 'Logro','Maravilla', 'Melódico', 'Motivación', 'Optimismo', 'Orgullo',
    'Positividad', 'Positivo', 'Reconfortante', 'Resiliencia', 'Resplandor', 'Reverencia', 'Romance', 'Satisfacción', 'Serenidad','Ternura', 'Triunfo', 'Ánimo', 'Éxito','Elación','Éxtasis']
print(f'Sentimientos positivos: {len(positivos)}'),

# 3. SENTIMIENTOS NEGATIVOS COMPLETOS (Dolor, ira, miedo, estrés, pérdida)
negativos = [
    'Abrumado', 'Aburrimiento', 'Aislamiento', 'Amargura', 'Angustia', 'Anhelo', 'Ansiedad', 'Aprensivo', 'Arrepentimiento', 'Asco',  'Decepción', 'Desamor', 'Desesperación', 'Despectivo', 'Devastado',
    'Dolor', 'Enojo', 'Entumecimiento', 'Envidia', 'Envidioso', 'Frustración', 'Frustrado', 'Lástima', 'Obstáculo', 'Malo', 'Melancolía', 'Miedo', 'Negativo', 'Odiar', 'Pena', 'Pérdida', 'Reflexión', 'Resentimiento', 'Soledad', 'Sufrimiento', 'Temeroso', 'Traición', 'Tristeza', 'Verguenza']
print(f'Sentimientos negativos: {len(negativos)}')

# 4. SENTIMIENTOS NEUTRALES (Estados ambiguos o contemplativos)
neutros = ['Ambivalencia', 'Curiosidad', 'Neutral','Sorpresa','Anticipación']
print(f'Sentimientos neutros: {len(neutros)}')

categorias = [positivos, negativos, neutros]

# Verificación del total
total_clasificados = len(positivos) + len(negativos) + len(neutros)
print(f'\n✅ Total clasificado: {total_clasificados}/106 sentimientos')
print(f'   - Positivos: {len(positivos)} ({len(positivos)/106*100:.1f}%)')
print(f'   - Negativos: {len(negativos)} ({len(negativos)/106*100:.1f}%)')
print(f'   - Neutros: {len(neutros)} ({len(neutros)/106*100:.1f}%)')
print(f'Total: {len(positivos) + len(negativos) + len(neutros)}')
print()

# Verificar si existen elementos en las listas que no se encuentran en la lista sentimientos_unicos
for sentimiento in negativos + positivos + neutros:
    if sentimiento not in sentimientos_unicos:
        print(f"❌ Sentimiento no encontrado en el dataset: {sentimiento}")

# Verificar si todos los sentimientos del dataset están clasificados
for sentimiento in sentimientos_unicos:
    if sentimiento not in positivos + negativos + neutros:
        print(f"❌ Sentimiento no clasificado: {sentimiento}")
else:
    print("✅ Todos los sentimientos del dataset están clasificados.")


Total de sentimientos únicos: 106
['Abrumado', 'Aburrimiento', 'Aceptación', 'Admiración', 'Adoración', 'Agradecido', 'Aislamiento', 'Alegría', 'Amabilidad', 'Amargura', 'Ambivalencia', 'Amistad', 'Amor', 'Angustia', 'Anhelo', 'Ansiedad', 'Anticipación', 'Apreciación', 'Aprensivo', 'Armonía', 'Arrepentimiento', 'Asco', 'Asombro', 'Cautivación', 'Celebración', 'Colorido', 'Confiado', 'Confianza', 'Contentamiento', 'Creatividad', 'Cumplimiento', 'Curiosidad', 'Decepción', 'Desamor', 'Descubrimiento', 'Desesperación', 'Deslumbrar', 'Despectivo', 'Determinación', 'Devastado', 'Disfrute', 'Diversión', 'Dolor', 'Elación', 'Elegancia', 'Emoción', 'Empoderamiento', 'Empático', 'Encantamiento', 'Energía', 'Enojo', 'Entumecimiento', 'Entusiasmo', 'Envidia', 'Envidioso', 'Esperanza', 'Euforia', 'Excitación', 'Felicidad', 'Frustración', 'Frustrado', 'Grandeza', 'Gratitud', 'Inspiración', 'Inspirado', 'Intimidación', 'Juguetón', 'Logro', 'Lástima', 'Malo', 'Maravilla', 'Melancolía', 'Melódico', 'Mi

#### **Función para categorizar sentimientos**

In [67]:
def categorizar_sentimiento(sentimiento, categorias):
    """
    Categoriza sentimientos solo si están en las listas definidas.
    Devuelve None para sentimientos no clasificados.
    """
    sent = str(sentimiento).strip().title()

    if sent in positivos:
        return 'positivo'
    elif sent in negativos:
        return 'negativo'
    elif sent in neutros:
        return 'neutral'
    else:
        # Devolvemos None para posterior filtrado
        return None


In [68]:
df1_clean.sample(3)

,texto,sentimiento,Texto_Limpio
61,Al experimentar la emoción de una carrera de F...,Emoción,Al experimentar la emocion de una carrera de F...
278,"En el torneo de tenis, un jugador muy esperado...",Frustración,"En el torneo de tenis, un jugador muy esperado..."
617,Rockeando con los solos de guitarra en un conc...,Neutral,Rockeando con los solos de guitarra en un conc...


In [69]:
df2_clean.sample(3)

,texto,sentimiento,Texto_Limpio
1110,Salgo re cansado del laburo pero llegar a casa...,negativo,Salgo re cansado del laburo pero llegar a casa...
2467,si nunca te dijieron que el auto esta re atrev...,positivo,si nunca te dijieron que el auto esta re atrev...
1798,Eso me pasa por desorganizado… Ah pero también...,neutral,Eso me pasa por desorganizado… Ah pero tambien...


#### **Categorizar sentimientos**

In [70]:
df1_clean['Sentimiento_Final'] = df1_clean['sentimiento'].apply(
    lambda x: categorizar_sentimiento(x,categorias)
)

df1_categorized = df1_clean[df1_clean['Sentimiento_Final'].notna()].copy()

df2_clean['Sentimiento_Final'] = df2_clean['sentimiento'].apply(
    lambda x: categorizar_sentimiento(x,categorias)
)

df2_categorized = df2_clean[df2_clean['Sentimiento_Final'].notna()].copy()

print(f"✅ df1: {len(df1_categorized)} registros categorizados")
print(f"✅ df2: {len(df2_categorized)} registros categorizados")

✅ df1: 732 registros categorizados
✅ df2: 2540 registros categorizados


In [71]:
df1_clean.sample(3)

,texto,sentimiento,Texto_Limpio,Sentimiento_Final
724,Vagando por el cementerio de los sueños perdid...,Soledad,Vagando por el cementerio de los sueños perdid...,negativo
699,Un espíritu libre que vuela en las alas de los...,Neutral,Un espiritu libre que vuela en las alas de los...,neutral
186,Decidió aprender un nuevo instrumento.Día uno:...,Frustración,Decidio aprender un nuevo instrumento.Dia uno:...,negativo


In [72]:
df2_clean.sample(3)

,texto,sentimiento,Texto_Limpio,Sentimiento_Final
613,incomprendido,negativo,incomprendido,negativo
352,estoy muy mayor he salido dolorido de los coch...,negativo,estoy muy mayor he salido dolorido de los coch...,negativo
1950,"“Tienes que volver a creer en ti, tú siempre h...",neutral,"“Tienes que volver a creer en ti, tu siempre h...",neutral


### <font color=lightgreen size=12>Limpiar dataset unificado</font>

#### **Función limpieza dataset unificado**

In [73]:
def limpiar_dataset_unificado(data, verbose=True):
    """
    Limpia dataset unificado para análisis de sentimientos.

    Proceso:
    1. Identifica y elimina CONTRADICCIONES (textos con diferentes sentimientos)
    2. Elimina DUPLICADOS exactos (mismo texto, mismo sentimiento)
    3. Limpieza final (espacios vacíos, NaN)

    Args:
        data: DataFrame con 'Texto_Limpio' y 'Sentimiento_Final'
        verbose: Si True, muestra análisis detallado

    Returns:
        DataFrame limpio, sin duplicados ni contradicciones
    """

    if verbose:
        print("🧹 LIMPIANDO DATASET UNIFICADO")
        print("-" * 50)
        print(f"Registros iniciales: {len(data):,}")
        print(f"Textos únicos iniciales: {data['Texto_Limpio'].nunique():,}")

    # Hacer copia para no modificar original
    df = data.copy()

    # ===== 1. ELIMINAR CONTRADICCIONES (PRIMERO) =====
    if verbose:
        print(f"\n1. 🔍 BUSCANDO CONTRADICCIONES...")

    # Textos con más de un sentimiento diferente
    conteo_sentimientos = df.groupby('Texto_Limpio')['Sentimiento_Final'].nunique()
    textos_con_contradiccion = conteo_sentimientos[conteo_sentimientos > 1].index.tolist()

    if textos_con_contradiccion:
        if verbose:
            print(f"   ⚠️  Encontradas: {len(textos_con_contradiccion):,} contradicciones")

            # Mostrar algunos ejemplos
            print(f"   • Ejemplos (primeros 2):")
            for texto in textos_con_contradiccion[:2]:
                sentimientos = df[df['Texto_Limpio'] == texto]['Sentimiento_Final'].unique()
                texto_corto = texto[:60] + "..." if len(texto) > 60 else texto
                print(f"     - '{texto_corto}'")
                print(f"       → Sentimientos: {', '.join(sentimientos)}")

        # Eliminar TODOS los registros de textos contradictorios
        df_sin_contradicciones = df[~df['Texto_Limpio'].isin(textos_con_contradiccion)].copy()

        if verbose:
            eliminados = len(df) - len(df_sin_contradicciones)
            print(f"   🗑️  Eliminados: {eliminados:,} registros por contradicciones")
    else:
        if verbose:
            print(f"   ✅ No hay contradicciones")
        df_sin_contradicciones = df.copy()

    # ===== 2. ELIMINAR DUPLICADOS EXACTOS =====
    if verbose:
        print(f"\n2. 🔍 BUSCANDO DUPLICADOS EXACTOS...")

    # Contar duplicados exactos (mismo texto, mismo sentimiento)
    conteo_duplicados = df_sin_contradicciones['Texto_Limpio'].value_counts()
    textos_duplicados = conteo_duplicados[conteo_duplicados > 1].index.tolist()

    if textos_duplicados:
        if verbose:
            print(f"   ⚠️  Encontrados: {len(textos_duplicados):,} textos duplicados")

            # Calcular cuántos registros se eliminarán
            total_a_eliminar = sum([conteo_duplicados[t] - 1 for t in textos_duplicados])
            print(f"   • Registros a eliminar: {total_a_eliminar:,}")

        # Eliminar duplicados (mantener primera aparición)
        df_sin_duplicados = df_sin_contradicciones.drop_duplicates(
            subset=['Texto_Limpio'],
            keep='first'
        )

        if verbose:
            eliminados = len(df_sin_contradicciones) - len(df_sin_duplicados)
            print(f"   🗑️  Eliminados: {eliminados:,} registros duplicados")
    else:
        if verbose:
            print(f"   ✅ No hay duplicados exactos")
        df_sin_duplicados = df_sin_contradicciones.copy()

    # ===== 3. LIMPIEZA FINAL =====
    if verbose:
        print(f"\n3. 🧹 LIMPIEZA FINAL...")

    df_final = df_sin_duplicados.copy()

    # Filtrar solo columnas necesarias
    df_final = df_final[['Texto_Limpio', 'Sentimiento_Final']]

    # Eliminar textos vacíos o solo espacios
    textos_vacios_antes = len(df_final)
    df_final = df_final[df_final['Texto_Limpio'].str.strip() != ""]
    textos_vacios_eliminados = textos_vacios_antes - len(df_final)

    if verbose and textos_vacios_eliminados > 0:
        print(f"   • Textos vacíos eliminados: {textos_vacios_eliminados}")

    # Eliminar sentimientos NaN
    sentimientos_nan_antes = len(df_final)
    df_final = df_final[df_final['Sentimiento_Final'].notna()]
    sentimientos_nan_eliminados = sentimientos_nan_antes - len(df_final)

    if verbose and sentimientos_nan_eliminados > 0:
        print(f"   • Sentimientos NaN eliminados: {sentimientos_nan_eliminados}")

    # ===== 4. VERIFICACIÓN Y RESUMEN =====
    if verbose:
        print(f"\n4. ✅ VERIFICACIÓN FINAL")
        print(f"   • Registros finales: {len(df_final):,}")
        print(f"   • Textos únicos finales: {df_final['Texto_Limpio'].nunique():,}")

        # Verificar que cada texto aparece solo una vez
        if len(df_final) == df_final['Texto_Limpio'].nunique():
            print(f"   🎯 ¡Dataset 100% limpio! Cada texto aparece solo una vez")
        else:
            diferencia = len(df_final) - df_final['Texto_Limpio'].nunique()
            print(f"   ⚠️  ¡Problema! Hay {diferencia} duplicados")

        # Resumen
        print(f"\n" + "=" * 50)
        print("📊 RESUMEN DE LIMPIEZA")
        print("=" * 50)

        total_eliminados = (len(data) - len(df_final))
        porcentaje_eliminado = (total_eliminados / len(data)) * 100

        print(f"Registros iniciales: {len(data):,}")
        print(f"Registros finales: {len(df_final):,}")
        print(f"Total eliminados: {total_eliminados:,} ({porcentaje_eliminado:.1f}%)")

        # Distribución de sentimientos
        print(f"\n📈 DISTRIBUCIÓN FINAL DE SENTIMIENTOS:")
        distribucion = df_final['Sentimiento_Final'].value_counts()
        for sentimiento, count in distribucion.items():
            porcentaje = (count / len(df_final)) * 100
            print(f"   • {sentimiento}: {count:,} ({porcentaje:.1f}%)")

    return df_final


In [74]:
df1_categorized.sample(3)
df2_categorized.sample(3)

,texto,sentimiento,Texto_Limpio,Sentimiento_Final
2457,Ese Anthony es osado oeeee quiere hacer su pro...,positivo,Ese Anthony es osado oeeee quiere hacer su pro...,positivo
2334,les juro que trato de ser optimista,positivo,les juro que trato de ser optimista,positivo
758,"Un lado mío está creativo, con ideas y ganas d...",negativo,"Un lado mio esta creativo, con ideas y ganas d...",negativo


#### **Unificar datataset y limpieza**

In [75]:
print("=" * 70)
print("🔗 UNIFICANDO DATASETS CATEGORIZADOS")
print("=" * 70)

# Unificar los datasets categorizados
df_unificado = pd.concat([df1_categorized[['Texto_Limpio', 'Sentimiento_Final']], df2_categorized[['Texto_Limpio', 'Sentimiento_Final']]], ignore_index=True)

print(f"📦 Dataset unificado: {df_unificado.shape}")
print(f"   • Registros: {len(df_unificado):,}")
print(f"   • Textos únicos: {df_unificado['Texto_Limpio'].nunique():,}")


# %%
print("\n" + "=" * 70)
print("🧹 APLICANDO LIMPIEZA AL DATASET UNIFICADO")
print("=" * 70)

# Aplicar limpieza
df_final = limpiar_dataset_unificado(df_unificado, verbose=True)

🔗 UNIFICANDO DATASETS CATEGORIZADOS
📦 Dataset unificado: (3272, 2)
   • Registros: 3,272
   • Textos únicos: 2,849

🧹 APLICANDO LIMPIEZA AL DATASET UNIFICADO
🧹 LIMPIANDO DATASET UNIFICADO
--------------------------------------------------
Registros iniciales: 3,272
Textos únicos iniciales: 2,849

1. 🔍 BUSCANDO CONTRADICCIONES...
   ⚠️  Encontradas: 90 contradicciones
   • Ejemplos (primeros 2):
     - '"De manera apacible, se puede sacudir el mundo" MG'
       → Sentimientos: negativo, positivo
     - '"He aprendido que el valor no es la ausencia de miedo, sino ...'
       → Sentimientos: neutral, positivo
   🗑️  Eliminados: 212 registros por contradicciones

2. 🔍 BUSCANDO DUPLICADOS EXACTOS...
   ⚠️  Encontrados: 252 textos duplicados
   • Registros a eliminar: 301
   🗑️  Eliminados: 301 registros duplicados

3. 🧹 LIMPIEZA FINAL...

4. ✅ VERIFICACIÓN FINAL
   • Registros finales: 2,759
   • Textos únicos finales: 2,759
   🎯 ¡Dataset 100% limpio! Cada texto aparece solo una vez

📊 RESU

In [76]:
df_unificado.sample(3)

,Texto_Limpio,Sentimiento_Final
3175,"A ver, si tan valiente te reto a etiquetar a l...",positivo
2956,Estoy bien,positivo
1470,El prejuicio es una carga que confunde el pasa...,negativo


 ### <font size=12 color=lightgreen>Análisis de Distribución y Visualización</font>

#### **Análisis de distribución de sentimientos**

In [77]:
#📊 ANÁLISIS DE DISTRIBUCIÓN DEL DATASET FINAL

print("=" * 60)
print("📈 ANÁLISIS DE DISTRIBUCIÓN - DATASET FINAL")
print("=" * 60)

# 1. Calcular conteos y porcentajes
conteos = df_final['Sentimiento_Final'].value_counts()
total_registros = len(df_final)
porcentajes = (conteos / total_registros * 100).round(2)

# 2. Mostrar tabla detallada
print(f"{'SENTIMIENTO':<12} | {'CANTIDAD':>8} | {'PORCENTAJE':>10} | {'PROPORCIÓN'}")
print("-" * 50)

for sentimiento in ['positivo', 'negativo', 'neutral']:
    if sentimiento in conteos:
        count = conteos[sentimiento]
        porcentaje = porcentajes[sentimiento]
        # Crear barra visual
        barra = '█' * int(count / total_registros * 40)  # Escala a 40 caracteres
        print(f"{sentimiento.capitalize():<12} | {count:>8} | {porcentaje:>9}% | {barra}")

print("-" * 50)
print(f"{'TOTAL':<12} | {total_registros:>8} | {'100.00':>9}% | {'█' * 40}")
print("-" * 58)


📈 ANÁLISIS DE DISTRIBUCIÓN - DATASET FINAL
SENTIMIENTO  | CANTIDAD | PORCENTAJE | PROPORCIÓN
--------------------------------------------------
Positivo     |     1177 |     42.66% | █████████████████
Negativo     |     1091 |     39.54% | ███████████████
Neutral      |      491 |      17.8% | ███████
--------------------------------------------------
TOTAL        |     2759 |    100.00% | ████████████████████████████████████████
----------------------------------------------------------


#### **Visualización de la distribución de Sentimientos**

In [78]:
# Grafica de pastel con Plotly

valores = df_final['Sentimiento_Final'].value_counts().reset_index()
valores.columns = ['sentimientos', 'Cantidad']
fig1 = px.pie(
    names = valores.sentimientos,
    values = valores.Cantidad,
)

fig1.update_traces(textposition='inside', textinfo='label+percent',  insidetextfont=dict(color = 'white', size=14)
)

fig1.update_layout(
    title_text=f'<b>Distribución de Sentimientos</b><br><span style="font-size:14px">Dataset Final: {total_registros} registros</span>',
    title_x=0.5,
    width=500,
    height=500,
    showlegend=False,
)

fig1.show()

### <font size=12 color=lightgreen> Exportar dataset </font>

#### **Definir ruta de exportación**

In [79]:
# Ruta actual
ruta_actual = Path.cwd()

# Buscar data-science
if ruta_actual.name == 'notebooks':
    # Si estamos en notebooks/, ir a ../datasets
    carpeta_datasets = ruta_actual.parent / 'datasets'
else:
    # Buscar data-science en directorios padres
    for directorio_padre in ruta_actual.parents:
        if (directorio_padre / 'data-science').exists():
            carpeta_datasets = directorio_padre / 'data-science' / 'datasets'
            break
    else:
        # Si no encuentra, usar directorio actual/datasets
        carpeta_datasets = ruta_actual / 'datasets'

# Crear carpeta si no existe
carpeta_datasets.mkdir(parents=True, exist_ok=True)

# Ruta completa del archivo
archivo_final = carpeta_datasets / 'dataset.csv'


#### **Exportar dataset**

In [80]:
# Renombrar columnas para formato final
df_exportar = df_final.rename({
    'Texto_Limpio': 'texto',
    'Sentimiento_Final': 'sentimiento'
}, axis=1)


metadata = {
    "total_registros": len(df_exportar),
    "distribucion": dict(df_exportar['sentimiento'].value_counts()),
    "fecha_creacion": datetime.now().isoformat(),
    "version": "1.0.0",
    "fuentes": [
        "sentimentdataset_es.csv",
        "sentiment_analysis_dataset.csv"
    ]
}

# Exportar
df_exportar.to_csv(archivo_final, index=False, encoding='utf-8-sig')
print(f"✅ Dataset exportado: {archivo_final}")
print(f"📊 Registros: {len(df_exportar):,}")

# Crear copia para trabajo posterior
df = df_exportar.copy()

✅ Dataset exportado: /content/datasets/dataset.csv
📊 Registros: 2,759


#### **Verificar exportación**

In [81]:
def verificar_csv_simple(ruta_archivo, mostrar_muestra=True):
    """
    Verificación simplificada con detección de encoding
    Y verificación de integridad mejorada
    """
    ruta = Path(ruta_archivo)

    if not ruta.exists():
        print(f"❌ Archivo no encontrado: {ruta}")
        return None

    # Detectar encoding
    encodings = ['utf-8-sig', 'utf-8', 'latin1', 'cp1252']

    for enc in encodings:
        try:
            # Probar con 5 filas primero
            df_test = pd.read_csv(ruta, encoding=enc, nrows=5)

            # Si llegamos aquí, el encoding funciona
            try:
                # Ahora cargar completo
                df = pd.read_csv(ruta, encoding=enc)
                print(f"✅ CSV cargado: {len(df):,} registros (encoding: {enc})")

                # 🔍 VERIFICACIÓN DE INTEGRIDAD MEJORADA
                print("🔍 Verificación de integridad:")
                print(f"   • Valores nulos totales: {df.isnull().sum().sum()}")
                print(f"   • Textos vacíos: {(df['texto'].str.strip() == '').sum()}")

                # Verificar que todos los sentimientos sean válidos
                sentimientos_validos = ['positivo', 'negativo', 'neutral']
                sentimientos_invalidos = df[~df['sentimiento'].isin(sentimientos_validos)]

                if len(sentimientos_invalidos) > 0:
                    print(f"   ⚠️  Sentimientos inválidos: {len(sentimientos_invalidos)}")
                    print(f"      Valores únicos inválidos: {sentimientos_invalidos['sentimiento'].unique()}")
                else:
                    print(f"   ✅ Todos los sentimientos son válidos")

                # Verificar unicidad
                textos_unicos = df['texto'].nunique()
                if len(df) == textos_unicos:
                    print(f"   ✅ 100% textos únicos: {textos_unicos:,} textos únicos")
                else:
                    print(f"   ⚠️  Duplicados: {len(df) - textos_unicos:,} textos duplicados")

                if mostrar_muestra:
                    print(f"📝 Columnas: {list(df.columns)}")
                    print(f"📊 Muestra (2 filas):")
                    print(df.head(2).to_string(index=False))

                return df

            except Exception as e:
                print(f"❌ Error cargando con encoding {enc}: {type(e).__name__}")
                continue

        except UnicodeDecodeError:
            continue

    print("❌ No se pudo cargar con ningún encoding común")
    return None

In [82]:
# Uso simple - así debería funcionar
df_check = verificar_csv_simple(archivo_final, mostrar_muestra=True)


✅ CSV cargado: 2,759 registros (encoding: utf-8-sig)
🔍 Verificación de integridad:
   • Valores nulos totales: 0
   • Textos vacíos: 0
   ✅ Todos los sentimientos son válidos
   ✅ 100% textos únicos: 2,759 textos únicos
📝 Columnas: ['texto', 'sentimiento']
📊 Muestra (2 filas):
                                           texto sentimiento
    ¡Acabo de adoptar a un lindo amigo peludo!??    positivo
¡Acabo de terminar un entrenamiento increible!??    positivo


In [83]:
# Verificar que el archivo se pueda leer
def verificar_csv_simple(ruta_archivo, mostrar_muestra=True):
    """
    Verificación simplificada con detección de encoding
    """
    ruta = Path(ruta_archivo)

    if not ruta.exists():
        print(f"❌ Archivo no encontrado: {ruta}")
        return None

    # Detectar encoding
    encodings = ['utf-8-sig', 'utf-8', 'latin1', 'cp1252']

    for enc in encodings:
        try:
            df = pd.read_csv(ruta, encoding=enc, nrows=5)  # Probar con 5 filas
            # Si llegamos aquí, el encoding funciona
            try:
                # Ahora cargar completo
                df = pd.read_csv(ruta, encoding=enc)
                print(f"✅ CSV cargado: {len(df):,} registros (encoding: {enc})")

                if mostrar_muestra:
                    print(f"📝 Columnas: {list(df.columns)}")
                    print(f"📊 Muestra (2 filas):")
                    print(df.head(2).to_string(index=False))

                return df
            except Exception as e:
                print(f"❌ Error cargando con encoding {enc}: {type(e).__name__}")
                continue

        except UnicodeDecodeError:
            continue

    print("❌ No se pudo cargar con ningún encoding común")
    return None

# Uso simple
df_check = verificar_csv_simple(archivo_final, mostrar_muestra=True)

✅ CSV cargado: 2,759 registros (encoding: utf-8-sig)
📝 Columnas: ['texto', 'sentimiento']
📊 Muestra (2 filas):
                                           texto sentimiento
    ¡Acabo de adoptar a un lindo amigo peludo!??    positivo
¡Acabo de terminar un entrenamiento increible!??    positivo


 ### <font size=12 color=lightgreen> Resumen ejecutivo </font>

In [84]:
print("=" * 60)
print("📋 RESUMEN EJECUTIVO - PREPROCESAMIENTO COMPLETADO")
print("=" * 60)
print(f"✅ Dataset final: {len(df_exportar)} registros")
print(f"✅ Distribución balanceada: Positivo {porcentajes['positivo']}%, Negativo {porcentajes['negativo']}%, Neutral {porcentajes['neutral']}%")
print(f"✅ Archivo exportado: {archivo_final}")
print(f"✅ Calidad: 0 textos vacíos, 0 valores nulos")
print("=" * 60)

📋 RESUMEN EJECUTIVO - PREPROCESAMIENTO COMPLETADO
✅ Dataset final: 2759 registros
✅ Distribución balanceada: Positivo 42.66%, Negativo 39.54%, Neutral 17.8%
✅ Archivo exportado: /content/datasets/dataset.csv
✅ Calidad: 0 textos vacíos, 0 valores nulos


---
### <font size=12 color=lightgreen>Observaciones</font>


### 1. **<font color='lightgreen'>Origen de los datos</font>**

Con el objetivo de mejorar la capacidad de generalización del modelo, se trabajó con dos datasets independientes obtenidos desde Kaggle.
Si bien ambos conjuntos de datos abordan el análisis de sentimiento en español, presentan diferencias en estructura, calidad lingüística y formato de origen. Su integración permitió ampliar la diversidad de expresiones textuales, reduciendo el sesgo hacia un único estilo de redacción y fortaleciendo la robustez del pipeline de preparación de datos en escenarios similares a producción.

#### **Fuentes de datos (Kaggle):**

https://www.kaggle.com/datasets/engineercolsoquas/spanish-sentiment-analysis-dataset

https://www.kaggle.com/datasets/kashishparmar02/social-media-sentiments-analysis-dataset


---
### 2. **<font color='lightgreen'> Informe de Desafíos Técnicos y Soluciones</font>**

#### **Dataset** 1 – Inconsistencias en el idioma

- Problema: El dataset original presentaba traducciones incompletas, combinando registros en español con fragmentos en su idioma original, además de traducciones literales de baja calidad. Esta situación afectaba la coherencia semántica del texto y podía introducir ruido en el análisis de sentimiento.

- Solución aplicada: Se utilizó la herramienta de Traducción de Microsoft Excel como apoyo para identificar registros no traducidos. No obstante, la corrección se realizó de forma manual y supervisada, revisando y ajustando cada registro individualmente con el fin de preservar el significado original del texto y evitar distorsiones semánticas. Posteriormente, se realizó una revisión manual (sanity check) para asegurar la consistencia lingüística del dataset completo.

- Impacto en el análisis: La normalización del idioma permitió obtener un corpus coherente en español, reduciendo ambigüedades y mejorando la calidad de los datos de entrada para la etapa de clasificación de sentimiento.


**Dataset 2 – Problemas de codificación de caracteres (encoding)**

- Problema:
El segundo dataset se encontraba en formato Excel y presentaba errores de codificación al ser abierto, evidenciados por la aparición de caracteres especiales incorrectos (mojibake), lo que impedía un procesamiento adecuado del texto.

- Solución aplicada:
Como primer paso, el archivo fue exportado a formato CSV. Posteriormente, se realizó la ingesta mediante Power Query, donde se configuró explícitamente la codificación Unicode (UTF-8), corrigiendo la estructura de caracteres antes de su integración al pipeline de preparación de datos.

- Impacto en el análisis:
La corrección del encoding aseguró la correcta interpretación de caracteres propios del idioma español, evitando pérdidas de información y mejorando la calidad del texto procesado.
---


### 3. **<font color='lightgreen'>Normalización y Limpieza de Texto</font>**
- Se aplicó una función de preprocesamiento (limpiar_texto_sentimiento) que incluyó:

- Preservación de mayúsculas/minúsculas (para mantener intensidad emocional).

- Eliminación de tildes (pero conservación de ñ/Ñ).

- Limpieza de URLs, menciones y caracteres no imprimibles.

- Normalización de espacios y saltos de línea.

**Nota: Se decidió no convertir todo a minúsculas para conservar pistas contextuales (ej. “¡GENIAL!” vs. “genial”), relevantes para modelos basados en intensidad emocional.**

---

### 4. <font color='lightgreen'>**Categorización de Sentimientos**</font>
Dado que el Dataset 1 contenía 106 sentimientos diferentes, se definió un esquema de agrupación en tres categorías:

Categoría	Ejemplos de Sentimientos Incluidos

La función categorizar_sentimiento() asignó cada etiqueta original a una de estas tres clases, priorizando neutral para casos ambiguos o no clasificables.

![image.png](attachment:image.png)

###  4. <font color='lightgreen'>**Limpieza Rigurosa del Dataset Unificado**</font>

**Problemas Identificados Post-Fusión**:
- **90 casos de contradicciones**: Textos idénticos con sentimientos diferentes (ej: "me siento bien" → positivo Y negativo)
- **252 casos de duplicados exactos**: Textos idénticos con mismo sentimiento (ej: "qué bonito día" → positivo, repetido)
- **Inconsistencia crítica** para entrenamiento de ML: Un texto no puede tener múltiples sentimientos

**Pipeline de 3 Etapas de Depuración**:

1. **Eliminación de Contradicciones** (prioridad máxima):
   - **90 textos problemáticos** identificados
   - **212 registros eliminados** (todos los registros de textos contradictorios)
   - **Ejemplo**: Texto "La vida es bella" aparecía 3 veces: 2×positivo, 1×negativo → se eliminaron LAS 3 apariciones

2. **Eliminación de Duplicados Exactos**:
   - **252 textos duplicados** identificados  
   - **301 registros eliminados** (se mantuvo solo la primera aparición de cada texto)
   - **Ejemplo**: "Hoy es mi cumpleaños" aparecía 4 veces como positivo → se mantuvo 1, se eliminaron 3

3. **Verificación Final de Consistencia**:
   - **2,759 textos únicos** (0% duplicados)
   - **0 contradicciones** (cada texto con un único sentimiento)
   - **2,759 registros finales** (cada texto aparece exactamente una vez)

**Métricas de Depuración**:
| Concepto | Cantidad | Explicación |
|----------|----------|-------------|
| **Textos iniciales** | 3,272 registros | Combinación cruda de ambos datasets |
| **Casos de contradicción** | 90 textos | Mismo texto, sentimientos diferentes |
| **Registros por contradicciones** | 212 eliminados | Todos los registros de textos contradictorios |
| **Casos de duplicación** | 252 textos | Mismo texto, mismo sentimiento |
| **Registros por duplicados** | 301 eliminados | Registros repetidos (manteniendo primero) |
| **Textos finales únicos** | 2,759 textos | 0 duplicados, 0 contradicciones |
| **Registros finales** | 2,759 registros | Un registro por texto único |
| **Tasa de retención** | 84.3% | 2,759/3,272 registros válidos |
| **Tasa de depuración** | 15.7% | 513/3,272 registros eliminados |

**Impacto en Calidad del Dataset**:
- ✅ **Consistencia absoluta**: Cada texto → un único sentimiento
- ✅ **Unicidad garantizada**: Sin repeticiones que inflen métricas
- ✅ **Preparado para ML**: Estructura óptima para entrenamiento y validación

---
### 5. **<font color='lightgreen'>Estructura final de Dataset Unificado</font>**

El dataset exportado ``dataset_listo_para_ML.csv`` contiene:

**Columnas:** texto, sentimiento

**Estadísticas finales**

Registros totales: 3,272

Distribución:

- Negativo: 1,300 (39.7%)

- Positivo: 1,231 (37.6%)

- Neutral: 741 (22.7%)`
---
---

In [85]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2759 entries, 0 to 3270
Data columns (total 2 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   texto        2759 non-null   object
 1   sentimiento  2759 non-null   object
dtypes: object(2)
memory usage: 64.7+ KB


 ### <font size=12 color=lightgreen> Machine Learning</font>

In [86]:
df

,texto,sentimiento
0,¡Acabo de adoptar a un lindo amigo peludo!??,positivo
1,¡Acabo de terminar un entrenamiento increible!??,positivo
2,¡Adoracion desbordante por un lindo cachorro r...,positivo
3,"¡Año nuevo, nuevos objetivos de fitness!??",positivo
4,¡Celebrando el cumpleaños de un amigo esta noc...,positivo
...,...,...
3266,Debes amar sin miedo a ser traicionado,positivo
3267,No podemos vivir con miedo: ¡Manejen borrachos...,positivo
3268,"La vida es un constante, SIN MIEDO AL EXITO ????",positivo
3269,Esquizofrenia = mente dividida: Miedo a las re...,positivo


In [87]:
import pandas as pd
import re
import nltk
from nltk.corpus import stopwords

# Descargar palabras vacías
nltk.download('stopwords')
stop_words = set(stopwords.words('spanish'))

def limpiar_texto(texto):
    texto = " ".join([word for word in texto.split() if word not in stop_words]) # Quitar stopwords
    return texto

# Aplicar a tu dataset
df['texto'] = df['texto'].apply(limpiar_texto)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [88]:
df

,texto,sentimiento
0,¡Acabo adoptar lindo amigo peludo!??,positivo
1,¡Acabo terminar entrenamiento increible!??,positivo
2,¡Adoracion desbordante lindo cachorro rescatad...,positivo
3,"¡Año nuevo, nuevos objetivos fitness!??",positivo
4,¡Celebrando cumpleaños amigo noche!??,positivo
...,...,...
3266,Debes amar miedo ser traicionado,positivo
3267,"No podemos vivir miedo: ¡Manejen borrachos, de...",positivo
3268,"La vida constante, SIN MIEDO AL EXITO ????",positivo
3269,Esquizofrenia = mente dividida: Miedo realidad...,positivo


 ### <font size=12 color=lightgreen> Balanceo del Dataset, TF-IDF, Modelo, Métricas y Serialización </font>

### Instalación de `imblearn`

Primero, necesitamos instalar la librería `imblearn`, que proporciona herramientas para manejar datasets desbalanceados, incluyendo la técnica SMOTE para sobremuestreo.

In [89]:
get_ipython().system('pip install imblearn')
print("Librería 'imblearn' instalada exitosamente.")

Librería 'imblearn' instalada exitosamente.


### Separación de Características y Target

Ahora, separaremos las características (el texto limpio) y la variable objetivo (el sentimiento) de nuestro DataFrame `df`. También mostraremos la distribución inicial de las clases para ver el desbalanceo.

In [90]:
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, accuracy_score, precision_score, recall_score, f1_score
import joblib
import json
import pandas as pd

# Separar características (X) y variable objetivo (y)
X = df['texto']
y = df['sentimiento']

# Verificar la distribución inicial de las clases
print("Distribución inicial de las clases:")
print(y.value_counts())

Distribución inicial de las clases:
sentimiento
positivo    1177
negativo    1091
neutral      491
Name: count, dtype: int64


### <font size=12 color=lightgreen>División de Datos (Entrenamiento y Prueba) y Vectorización TF-IDF</font>



Es crucial dividir el dataset en conjuntos de entrenamiento y prueba *antes* de aplicar SMOTE para evitar la fuga de datos (data leakage). Luego, transformaremos los textos en vectores numéricos usando `TfidfVectorizer`.

In [91]:
# Dividir el dataset en conjuntos de entrenamiento y prueba ANTES de aplicar SMOTE
X_train_unbalanced, X_test, y_train_unbalanced, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

print(f"\nTamaño del conjunto de entrenamiento (desbalanceado): {len(X_train_unbalanced)} muestras")
print(f"Tamaño del conjunto de prueba: {len(X_test)} muestras")
print(f"Distribución de clases en el conjunto de entrenamiento (desbalanceado):\n{y_train_unbalanced.value_counts()}")
print(f"Distribución de clases en el conjunto de prueba:\n{y_test.value_counts()}")

# Inicializar TfidfVectorizer
tfidf_vectorizer = TfidfVectorizer(max_features=5000,ngram_range=(1,2)) # Limitando las características para eficiencia

# Ajustar y transformar X_train_unbalanced, y transformar X_test
X_train_tfidf_unbalanced = tfidf_vectorizer.fit_transform(X_train_unbalanced)
X_test_tfidf = tfidf_vectorizer.transform(X_test)

print("\nVectorización TF-IDF completada en la división desbalanceada.")
print(f"Forma de X_train_tfidf_unbalanced: {X_train_tfidf_unbalanced.shape}")
print(f"Forma de X_test_tfidf: {X_test_tfidf.shape}")


Tamaño del conjunto de entrenamiento (desbalanceado): 2207 muestras
Tamaño del conjunto de prueba: 552 muestras
Distribución de clases en el conjunto de entrenamiento (desbalanceado):
sentimiento
positivo    941
negativo    873
neutral     393
Name: count, dtype: int64
Distribución de clases en el conjunto de prueba:
sentimiento
positivo    236
negativo    218
neutral      98
Name: count, dtype: int64

Vectorización TF-IDF completada en la división desbalanceada.
Forma de X_train_tfidf_unbalanced: (2207, 5000)
Forma de X_test_tfidf: (552, 5000)


### <font size=12 color=lightgreen> Balanceo del Conjunto de Entrenamiento con SMOTE</font>

Ahora aplicaremos SMOTE solo al conjunto de entrenamiento vectorizado (`X_train_tfidf_unbalanced`) para balancear las clases, generando muestras sintéticas para las clases minoritarias.

In [92]:
# Inicializar SMOTE para balancear el conjunto de datos de ENTRENAMIENTO
smote = SMOTE(random_state=42)
X_train_tfidf, y_train = smote.fit_resample(X_train_tfidf_unbalanced, y_train_unbalanced)

print("\nDistribución de clases después de SMOTE en los datos de entrenamiento:")
print(y_train.value_counts())

print(f"Forma de X_train_tfidf después de SMOTE: {X_train_tfidf.shape}")


Distribución de clases después de SMOTE en los datos de entrenamiento:
sentimiento
negativo    941
positivo    941
neutral     941
Name: count, dtype: int64
Forma de X_train_tfidf después de SMOTE: (2823, 5000)


### <font size=12 color=lightgreen> Entrenamiento del Modelo de Regresión Logística</font>



Entrenaremos un modelo de Regresión Logística utilizando los datos de entrenamiento balanceados y vectorizados.

In [93]:
# Entrenar el Modelo de Regresión Logística
model = LogisticRegression(max_iter=1000, random_state=42)
model.fit(X_train_tfidf, y_train)

print("\nModelo de Regresión Logística entrenado.")


Modelo de Regresión Logística entrenado.


### <font size=12 color=lightgreen>Evaluación del Modelo</font>

Evaluaremos el rendimiento del modelo en el conjunto de prueba utilizando métricas clave como accuracy, precision, recall y F1-score.

In [94]:
# Evaluar el Modelo
y_pred = model.predict(X_test_tfidf)
y_pred_proba = model.predict_proba(X_test_tfidf)

print("\nEvaluación del Modelo:")
print(f"Accuracy: {accuracy_score(y_test, y_pred):.2f}")
print(f"Precision (ponderada): {precision_score(y_test, y_pred, average='weighted'):.2f}")
print(f"Recall (ponderado): {recall_score(y_test, y_pred, average='weighted'):.2f}")
print(f"F1-Score (ponderado): {f1_score(y_test, y_pred, average='weighted'):.2f}")
print("\nReporte de Clasificación:\n", classification_report(y_test, y_pred))


Evaluación del Modelo:
Accuracy: 0.78
Precision (ponderada): 0.78
Recall (ponderado): 0.78
F1-Score (ponderado): 0.78

Reporte de Clasificación:
               precision    recall  f1-score   support

    negativo       0.82      0.84      0.83       218
     neutral       0.66      0.66      0.66        98
    positivo       0.80      0.78      0.79       236

    accuracy                           0.78       552
   macro avg       0.76      0.76      0.76       552
weighted avg       0.78      0.78      0.78       552



### <font size=12 color=lightgreen> Serialización del Modelo y Vectorizadors</font>



Guardaremos el modelo entrenado y el objeto `TfidfVectorizer` utilizando `joblib` para poder reutilizarlos más tarde en la API de predicción.

In [95]:
# Serializar el Modelo y el Vectorizador
joblib.dump(model, '/content/modelo_sentimientos.pkl')
joblib.dump(tfidf_vectorizer, '/content/vectorizador.pkl')

print("\nModelo y vectorizador guardados exitosamente en '/content/modelo_sentimientos.pkl' y '/content/vectorizador.pkl'.")


Modelo y vectorizador guardados exitosamente en '/content/modelo_sentimientos.pkl' y '/content/vectorizador.pkl'.


### Prueba del Modelo con Salida JSON

Crearemos una función para probar el modelo con nuevas reseñas de texto. Esta función preprocesará el texto, lo vectorizará con el `TfidfVectorizer` guardado, realizará una predicción y devolverá el resultado en formato JSON, incluyendo la previsión y la probabilidad de la clase predicha.

In [96]:
# Recargar el modelo y el vectorizador para probar (como si fuera una nueva sesión/API)
loaded_model = joblib.load('/content/modelo_sentimientos.pkl')
loaded_vectorizer = joblib.load('/content/vectorizador.pkl')

def predict_sentiment_json(text_review):
    # Preprocesamiento (igual que para los datos de entrenamiento)
    # Asumiendo que `pre_proccess_text` y `limpiar_texto` están definidos en celdas anteriores
    cleaned_text = limpiar_texto(text_review)
    cleaned_text = limpiar_texto(cleaned_text)

    # Vectorizar el texto limpio
    text_vectorized = loaded_vectorizer.transform([cleaned_text])

    # Predecir el sentimiento
    prediction = loaded_model.predict(text_vectorized)[0]

    # Predecir las probabilidades
    probabilities = loaded_model.predict_proba(text_vectorized)[0]
    class_labels = loaded_model.classes_
    # Asegurar el mapeo correcto de probabilidades a etiquetas
    prob_dict = {label: round(prob * 100, 2) for label, prob in zip(class_labels, probabilities)}

    # Obtener la probabilidad de la clase predicha
    predicted_prob = prob_dict[prediction]

    result = {
        "prevision": prediction,
        "probabilidad": predicted_prob
    }
    return json.dumps(result, indent=4)

# Ejemplos de uso de la función de predicción
new_review1 = "Tengo hambre"
new_review2 = "mala actitud del personal"
new_review3 = "La situación es complicada, no sé qué pensar."

print(f"\nPredicción para '{new_review1}':")
print(predict_sentiment_json(new_review1))

print(f"\nPredicción para '{new_review2}':")
print(predict_sentiment_json(new_review2))

print(f"\nPredicción para '{new_review3}':")
print(predict_sentiment_json(new_review3))


Predicción para 'Tengo hambre':
{
    "prevision": "negativo",
    "probabilidad": 34.5
}

Predicción para 'mala actitud del personal':
{
    "prevision": "positivo",
    "probabilidad": 55.58
}

Predicción para 'La situación es complicada, no sé qué pensar.':
{
    "prevision": "neutral",
    "probabilidad": 42.59
}


### <font size=12 color=lightgreen>Exportación del modelo</font>

In [97]:
from sklearn.pipeline import Pipeline
import joblib

# Creamos un Pipeline manual uniendo las dos piezas
pipeline_para_produccion = Pipeline([
    ('vectorizer', tfidf_vectorizer), # Primero transforma el texto a números
    ('classifier', model)             # Luego predice con esos números
])

# Probamos que funcione antes de exportar
test_text = ["Este es un ejemplo de prueba para ver si funciona el pipeline"]
prediccion = pipeline_para_produccion.predict(test_text)
print(f"Prueba del pipeline: {prediccion}")

# EXPORTAR EL ARCHIVO FINAL
# Este es el archivo que debes subir a la carpeta de tu microservicio
joblib.dump(pipeline_para_produccion, 'modelo_entrenado.joblib')

print("✅ Archivo 'modelo_entrenado.joblib' creado exitosamente.")

Prueba del pipeline: ['negativo']
✅ Archivo 'modelo_entrenado.joblib' creado exitosamente.
